In [1]:
#all imports

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as func
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType
from pyspark import SparkContext

In [2]:
# Create Spark session
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

### find average numb of friends of diff age group using SQL along with RDD

In [4]:
#Adding column names/header to the table/df, have to give data type too since file doesnot have header(schema define)

def mapper(line):
    fields = line.split(',')
    return Row(ID=int(fields[0]), name=str(fields[1]),age=int(fields[2]), numfriends=int(fields[3]))


#create rdd

rdd = spark.sparkContext.textFile("./data/fakefriends.csv")

# get required fields from rdd(using mapper func which gets req fields and give name to each col)

parsed_data = rdd.map(mapper)

## Infer the schema, and register the RDD as a dataframe : 

schemaPeople = spark.createDataFrame(parsed_data).cache()

## give df a name as table/create a table :

schemaPeople.createOrReplaceTempView("Friends")

# SQL can be run over DataFrames that have been registered as a table

teenagers = spark.sql("SELECT * FROM Friends WHERE age>=18 and age <=19")
teenagers.show()

#direct function on schema/dataframe (not in table)

schemaPeople.groupBy("age").avg("numfriends").show()
#spark.stop()



+---+-------+---+----------+
| ID|   name|age|numfriends|
+---+-------+---+----------+
| 21|  Miles| 19|       268|
| 52|Beverly| 19|       269|
| 54|  Brunt| 19|         5|
|106|Beverly| 18|       499|
|115|  Dukat| 18|       397|
|133|  Quark| 19|       265|
|136|   Will| 19|       335|
|225|   Elim| 19|       106|
|304|   Will| 19|       404|
|341|   Data| 18|       326|
|366|  Keiko| 19|       119|
|373|  Quark| 19|       272|
|377|Beverly| 18|       418|
|404| Kasidy| 18|        24|
|409|    Nog| 19|       267|
|439|   Data| 18|       417|
|444|  Keiko| 18|       472|
|492|  Dukat| 19|        36|
|494| Kasidy| 18|       194|
+---+-------+---+----------+

+---+------------------+
|age|   avg(numfriends)|
+---+------------------+
| 26|242.05882352941177|
| 29|215.91666666666666|
| 65|             298.2|
| 54| 278.0769230769231|
| 19|213.27272727272728|
| 22|206.42857142857142|
| 34|             245.5|
| 50|             254.6|
| 57| 258.8333333333333|
| 43|230.57142857142858|
| 32| 2

In [5]:
##for the files which already have header can create dataframe without creating rdd and no custom schema
#infers schema based on the data-values on the file

friends_df = spark.read.option("header","true").option("inferschema","true").\
csv("./data/fakefriends-header.csv")

friends_df.printSchema()

#can create table from df and perform SQL queries on that

friends_df.createOrReplaceTempView("friends_table")       

#find out average numb of friends of diff age group using SQL query on table

average_numfriends = spark.sql("SELECT age, COUNT(age) , SUM(friends), SUM(friends)/count(age) AS average_friends\
                                FROM friends_table GROUP BY age ORDER BY age")
average_numfriends.show()

#can call functions directly on dataframes which are different to SQL queries

friends_df.select("name").show(5)

#finds out num of peope of same name

friends_df.groupBy("name").count().orderBy("name").show()

#find out average numb of friends of diff age group using function on df
#agg-->aggregated multiple functions on specific column

friends_df.groupBy("age").agg(func.round(func.avg("friends"),2).alias("average_num_0f_friends"))\
.orderBy("age").show()

root
 |-- userID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)

+---+----------+------------+------------------+
|age|count(age)|sum(friends)|   average_friends|
+---+----------+------------+------------------+
| 18|         8|        2747|           343.375|
| 19|        11|        2346|213.27272727272728|
| 20|         5|         825|             165.0|
| 21|         8|        2807|           350.875|
| 22|         7|        1445|206.42857142857142|
| 23|        10|        2463|             246.3|
| 24|         5|        1169|             233.8|
| 25|        11|        2172|197.45454545454547|
| 26|        17|        4115|242.05882352941177|
| 27|         8|        1825|           228.125|
| 28|        10|        2091|             209.1|
| 29|        12|        2591|215.91666666666666|
| 30|        11|        2594| 235.8181818181818|
| 31|         8|        2138|            267.25|
| 32|     

In [6]:
##For files without header
###Adding column names/header to the df,have to give data type too since file doesnot have header(schema define)


temp_rdd = spark.sparkContext.textFile("./data/1800.csv")

def parse(lines):
    parsed_lines = lines.split(",")
    temp = parsed_lines[3]
    return Row(station_id=str(parsed_lines[0]), temp_type = str(parsed_lines[2]), temp = float(parsed_lines[3]))


##create rdd

parsed_temp = temp_rdd.map(parse)

##create dataframe

temp_df = spark.createDataFrame(parsed_temp).cache()
temp_df.printSchema()


##direct functions on df
##selecting only TMIN comprising df
TMIN_df = temp_df.filter(temp_df.temp_type == "TMIN")

Station_temp_df = TMIN_df.select("station_id","temp")


## finding minimum temperature of the distinct stations

Min_temp  = Station_temp_df.groupby("station_id").agg(func.min("temp").alias("minimum_temperatue"))

#adding new column to the df : changing temperature to Farheneit

Min_temp_Farh =  Min_temp.withColumn("TEMPFAR",\
                                    (func.round(func.col("minimum_temperatue") * 0.1*(9.0/5.0) +32.0,2))).\
                                    select("station_id","TEMPFAR")

for key , value in Min_temp_Farh.collect():
    print(key,value,"F")
    
Min_temp.withColumn("TEMPFAR",(func.round(func.col("minimum_temperatue") * 0.1*(9.0/5.0) +32.0,2))).\
select("station_id","TEMPFAR","minimum_temperatue").show()

root
 |-- station_id: string (nullable = true)
 |-- temp_type: string (nullable = true)
 |-- temp: double (nullable = true)

ITE00100554 5.36 F
EZE00100082 7.7 F
+-----------+-------+------------------+
| station_id|TEMPFAR|minimum_temperatue|
+-----------+-------+------------------+
|ITE00100554|   5.36|            -148.0|
|EZE00100082|    7.7|            -135.0|
+-----------+-------+------------------+



In [7]:

#give header to the data without creating RDD using data types and Structtype  imports from pyspark.sql.types

header = StructType([ StructField("customerid",IntegerType(),True),\
                        StructField("customer",IntegerType(),True),\
                        StructField("money_spent",FloatType(),True)
            ])

customer_order_df = spark.read.schema(header).csv("./data/customer-orders.csv")

#find out the total money spent by each customer

customer_order_df.groupBy("customerid")\
                        .agg(func.round(func.sum("money_spent"),2).alias("Total_money_spent")).\
                        orderBy("Total_money_spent").show(customer_order_df.count())

#total numb of rows

customer_order_df.count()

+----------+-----------------+
|customerid|Total_money_spent|
+----------+-----------------+
|        45|          3309.38|
|        79|          3790.57|
|        96|          3924.23|
|        23|          4042.65|
|        99|          4172.29|
|        75|           4178.5|
|        36|          4278.05|
|        98|          4297.26|
|        47|           4316.3|
|        77|          4327.73|
|        13|          4367.62|
|        48|          4384.33|
|        49|           4394.6|
|        94|          4475.57|
|        67|          4505.79|
|        50|          4517.27|
|        78|          4524.51|
|         5|          4561.07|
|        57|           4628.4|
|        83|           4635.8|
|        91|          4642.26|
|        74|          4647.13|
|        84|          4652.94|
|         3|          4659.63|
|        12|          4664.59|
|        66|          4681.92|
|        56|          4701.02|
|        21|          4707.41|
|        80|          4727.86|
|       

10000

In [8]:

schema = StructType([
    StructField("userId",IntegerType(),True),\
    StructField("movieId",IntegerType(),True),\
    StructField("rating",FloatType(),True),\
    StructField("timestamp",StringType(),True)\
])
    
movie_ratings = spark.read.option("header",True).schema(schema).csv("./data/ratings.csv")
movie_ratings.printSchema()

movie_ratings.groupBy("movieId").agg(func.sum("rating")).show(5)

popular_movie = movie_ratings.groupBy("movieId").agg(func.count("movieId").alias("Movie_id_count")).\
                orderBy(func.desc("Movie_id_count")).show(3)




root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)

+-------+-----------+
|movieId|sum(rating)|
+-------+-----------+
|   1580|      575.5|
|   2366|       91.0|
|   3175|      268.5|
|   1088|      141.5|
|  32460|       17.0|
+-------+-----------+
only showing top 5 rows

+-------+--------------+
|movieId|Movie_id_count|
+-------+--------------+
|    356|           329|
|    318|           317|
|    296|           307|
+-------+--------------+
only showing top 3 rows



In [9]:
#giving schema to the file

schema = StructType([
    StructField("userId",IntegerType(),True),\
    StructField("movieId",IntegerType(),True),\
    StructField("rating",FloatType(),True),\
    StructField("timestamp",StringType(),True)\
])
    
movie_ratings = spark.read.option("header",True).schema(schema)\
                .csv("./data/ratings.csv")

type(movie_ratings)
movie_ratings.printSchema()

#aggregate funcs such as sum,max,min require agg
movie_ratings.groupBy("movieId").agg(func.sum("rating")).show(5)


#aggregate function on multiple columns: find the most popular movie, movie which has been rated most num of times
popular_movie = movie_ratings.groupBy("movieId").agg(func.count("movieId").alias("Movie_id_count"),\
               func.sum("rating").alias("sum_of_ratings")).\
                orderBy(func.desc("Movie_id_count"))

#Popular movies with their average ratings

popular_movie_average_rating =  popular_movie.withColumn('Average_rating', \
                                popular_movie.sum_of_ratings/popular_movie.Movie_id_count)\
                                .select("movieId","Movie_id_count","Average_rating")
popular_movie_average_rating.show(3)


root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)

+-------+-----------+
|movieId|sum(rating)|
+-------+-----------+
|   1580|      575.5|
|   2366|       91.0|
|   3175|      268.5|
|   1088|      141.5|
|  32460|       17.0|
+-------+-----------+
only showing top 5 rows

+-------+--------------+-----------------+
|movieId|Movie_id_count|   Average_rating|
+-------+--------------+-----------------+
|    356|           329|4.164133738601824|
|    318|           317|4.429022082018927|
|    296|           307|4.197068403908795|
+-------+--------------+-----------------+
only showing top 3 rows



In [10]:
import codecs
def movie_mapping():
    movie_id_names = {}
    with open("./data/movies.csv","r",encoding='ISO-8859-1', errors='ignore') as namefile:
        for lines in namefile:
            line_list = lines.split(",")
            movie_id_names[line_list[0]] = line_list[1] 
    return movie_id_names

name_Dict = spark.sparkContext.broadcast(movie_mapping())
# Create a user-defined function to look up movie names from our broadcasted dictionary
def lookupName(movieID):
    return name_Dict.value[str(movieID)]


# Create a user-defined function to look up movie names from our broadcasted dictionary


lookupNameUDF = func.udf(lookupName)

# Add a movieTitle column using our new udf to df with movie-id,count ,average rating

moviesWithNames = popular_movie_average_rating.withColumn("movieTitle", lookupNameUDF(func.col("movieID")))


moviesWithNames.show()


+-------+--------------+------------------+--------------------+
|movieId|Movie_id_count|    Average_rating|          movieTitle|
+-------+--------------+------------------+--------------------+
|    356|           329| 4.164133738601824| Forrest Gump (1994)|
|    318|           317| 4.429022082018927|"Shawshank Redemp...|
|    296|           307| 4.197068403908795| Pulp Fiction (1994)|
|    593|           279| 4.161290322580645|"Silence of the L...|
|   2571|           278| 4.192446043165468|             "Matrix|
|    260|           251| 4.231075697211155|Star Wars: Episod...|
|    480|           238|              3.75|Jurassic Park (1993)|
|    110|           237| 4.031645569620253|   Braveheart (1995)|
|    589|           224| 3.970982142857143|Terminator 2: Jud...|
|    527|           220|             4.225|Schindler's List ...|
|   2959|           218| 4.272935779816514|   Fight Club (1999)|
|      1|           215|3.9209302325581397|    Toy Story (1995)|
|   1196|           211|4

In [11]:
#spiderman 

schema = StructType([StructField("superhero_id", IntegerType(),True),\
                  StructField("superhero_name", StringType(),True) ])
   
superhero_df = spark.read.schema(schema).option("sep"," ")\
                .csv("./data/Marvel+Names.txt")


superhero_graph_df = spark.read.text("./data/Marvel+Graph.txt")

connections = superhero_graph_df.withColumn("id", func.split(func.trim(func.col("value")), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.trim(func.col("value")), " ")) - 1) \
    .groupBy("id").agg(func.sum("connections").alias("connections"))

mostPopular = connections.sort(func.col("connections").desc()).first()

mostPopularName = superhero_df.filter(func.col("superhero_id") == mostPopular[0]).select("superhero_name").first()

print(mostPopularName[0] + " is the most popular superhero with " + str(mostPopular[1]) + " co-appearances.")




CAPTAIN AMERICA is the most popular superhero with 1933 co-appearances.


In [13]:
import csv

with codecs.open("./data/movies.csv", "r", encoding='ISO-8859-1') as namefile:
        csvreader_object=csv.reader(namefile)
            # The line will skip the first row of the csv file (Header row)
        next(csvreader_object)

            # We are now printing all rows except the first row of the csv
        for row in csvreader_object:
            print(row[1])           

Toy Story (1995)
Jumanji (1995)
Grumpier Old Men (1995)
Waiting to Exhale (1995)
Father of the Bride Part II (1995)
Heat (1995)
Sabrina (1995)
Tom and Huck (1995)
Sudden Death (1995)
GoldenEye (1995)
American President, The (1995)
Dracula: Dead and Loving It (1995)
Balto (1995)
Nixon (1995)
Cutthroat Island (1995)
Casino (1995)
Sense and Sensibility (1995)
Four Rooms (1995)
Ace Ventura: When Nature Calls (1995)
Money Train (1995)
Get Shorty (1995)
Copycat (1995)
Assassins (1995)
Powder (1995)
Leaving Las Vegas (1995)
Othello (1995)
Now and Then (1995)
Persuasion (1995)
City of Lost Children, The (CitÃ© des enfants perdus, La) (1995)
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
Dangerous Minds (1995)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Babe (1995)
Dead Man Walking (1995)
It Takes Two (1995)
Clueless (1995)
Cry, the Beloved Country (1995)
Richard III (1995)
Dead Presidents (1995)
Restoration (1995)
Mortal Kombat (1995)
To Die For (1995)
How to Make an American Quilt (1995)


Last Days, The (1998)
Fantastic Planet, The (PlanÃ¨te sauvage, La) (1973)
Blast from the Past (1999)
Message in a Bottle (1999)
My Favorite Martian (1999)
Jawbreaker (1999)
October Sky (1999)
Office Space (1999)
Apple, The (Sib) (1998)
200 Cigarettes (1999)
8MM (1999)
Other Sister, The (1999)
Long Goodbye, The (1973)
Ballad of Narayama, The (Narayama bushiko) (1983)
Pet Sematary (1989)
Pet Sematary II (1992)
Children of the Corn II: The Final Sacrifice (1993)
Children of the Corn III (1994)
Christine (1983)
Night Shift (1982)
Airport (1970)
Airport 1975 (1974)
Airport '77 (1977)
Rollercoaster (1977)
Towering Inferno, The (1974)
Alligator (1980)
Meteor (1979)
Westworld (1973)
Logan's Run (1976)
Planet of the Apes (1968)
Beneath the Planet of the Apes (1970)
Battle for the Planet of the Apes (1973)
Conquest of the Planet of the Apes (1972)
Escape from the Planet of the Apes (1971)
Avalanche (1978)
Earthquake (1974)
Beyond the Poseidon Adventure (1979)
Dancemaker (1998)
Analyze This (1999

Eddie and the Cruisers (1983)
Erik the Viking (1989)
Family Business (1989)
Farewell to the King (1989)
Fat Man and Little Boy (1989)
Gleaming the Cube (1989)
Great Balls of Fire! (1989)
Gross Anatomy (a.k.a. A Cut Above) (1989)
Harlem Nights (1989)
Her Alibi (1989)
How to Get Ahead in Advertising (1989)
Innocent Man, An (1989)
January Man, The (1989)
Johnny Handsome (1989)
Jesus of Montreal (JÃ©sus de MontrÃ©al) (1989)
K-9 (1989)
Kickboxer (1989)
Last Exit to Brooklyn (1989)
Lean on Me (1989)
Let It Ride (1989)
Little Monsters (1989)
Look Who's Talking (1989)
Loverboy (1989)
Major League (1989)
Meet the Feebles (1989)
Millennium (1989)
Miracle Mile (1989)
New York Stories (1989)
Next of Kin (1989)
No Holds Barred (1989)
Package, The (1989)
Penn & Teller Get Killed (1989)
Pink Cadillac (1989)
Punisher, The (1989)
Jurassic Park III (2001)
America's Sweethearts (2001)
Brother (2000)
Ghost World (2001)
Hedwig and the Angry Inch (2000)
Planet of the Apes (2001)
Bread and Tulips (Pane e tul

Allegro non troppo (1977)
Black Widow (1987)
Herod's Law (Ley de Herodes, La) (2000)
Flesh & Blood (1985)
King Solomon's Mines (1985)
Raw Deal (1986)
Goodbye, Mr. Chips (1939)
Follow Me, Boys! (1966)
Against the Ropes (2004)
Confessions of a Teenage Drama Queen (2004)
EuroTrip (2004)
Passion of the Christ, The (2004)
Club Dread (2004)
Dirty Dancing: Havana Nights (2004)
Twisted (2004)
Good bye, Lenin! (2003)
Hidalgo (2004)
Starsky & Hutch (2004)
Reckoning, The (2004)
Persona (1966)
Passion of Anna, The (Passion, En) (1969)
Corbeau, Le (Raven, The) (1943)
Pickup on South Street (1953)
Damned, The (La Caduta degli dei) (1969)
Just One of the Guys (1985)
Wrong Arm of the Law, The (1963)
Agent Cody Banks 2: Destination London (2004)
Girl Next Door, The (2004)
Secret Window (2004)
Spartan (2004)
Broken Wings (Knafayim Shvurot) (2002)
Wilbur Wants to Kill Himself (2002)
Mad Dog and Glory (1993)
Peyton Place (1957)
Dawn of the Dead (2004)
Eternal Sunshine of the Spotless Mind (2004)
Taking Li

Notes on a Scandal (2006)
DOA: Dead or Alive (2006)
Curse of the Golden Flower (Man cheng jin dai huang jin jia) (2006)
Good German, The (2006)
Letters from Iwo Jima (2006)
Black Christmas (2006)
Stomp the Yard (2007)
Miss Potter (2006)
American Pie Presents The Naked Mile (American Pie 5: The Naked Mile) (2006)
Venus (2006)
The Return of Don Camillo (1953)
The Little World of Don Camillo (1952)
Primeval (2007)
Alpha Dog (2007)
Hitcher, The (2007)
After the Wedding (Efter brylluppet) (2006)
Bridge to Terabithia (2007)
Beer League (2006)
Dead Meat (2004)
49 Up (2005)
Waitress (2007)
Seven Up! (1964)
Catch and Release (2006)
Smokin' Aces (2006)
Blood and Chocolate (2007)
Epic Movie (2007)
Messengers, The (2007)
Because I Said So (2007)
Hannibal Rising (2007)
Norbit (2007)
Boss of It All, The (DirektÃ¸ren for det hele) (2006)
Cocaine Cowboys (2006)
Ratatouille (2007)
Paris, I Love You (Paris, je t'aime) (2006)
Astronaut Farmer, The (2007)
Wake Up, Ron Burgundy (2004)
It's a Boy Girl Thing

Five-Year Engagement, The (2012)
Think Like a Man (2012)
Lucky One, The (2012)
Safe (2012)
Dark Shadows (2012)
96 Minutes (2011) 
Decoy Bride, The (2011)
Rocket Singh: Salesman of the Year (2009)
Dictator, The (2012)
Walking with Monsters (2005)
Men in Black III (M.III.B.) (M.I.B.Â³) (2012)
Snow White and the Huntsman (2012)
Sound of My Voice (2011)
Eva (2011)
Pirates! Band of Misfits, The (2012)
Prometheus (2012)
Pact, The (2012)
Bernie (2011)
Inhale (2010)
Take This Waltz (2011)
Wanderlust (2012)
Moonrise Kingdom (2012)
Get the Gringo (2012)
Superman/Doomsday (2007) 
Thousand Words, A (2012)
Safety Not Guaranteed (2012)
Madagascar 3: Europe's Most Wanted (2012)
Your Sister's Sister (2011)
Dragon Ball: The Curse Of The Blood Rubies (Doragon bÃ´ru: Shenron no densetsu) (1986)
Dragon Ball: Sleeping Princess in Devil's Castle (Doragon bÃ´ru: MajinjÃ´ no nemuri hime) (1987)
Superman/Batman: Public Enemies (2009)
Dragon Ball: Mystical Adventure (Doragon bÃ´ru: Makafushigi dai bÃ´ken) (1988

Miss Peregrine's Home for Peculiar Children (2016)
The Girl on the Train (2016)
The Accountant (2016)
Imperium (2016)
Kizumonogatari Part 1: Tekketsu (2016)
Steins;Gate the Movie: The Burden of DÃ©jÃ  vu (2013)
Shin Godzilla (2016)
Winnie Pooh (1969)
Winnie the Pooh Goes Visiting (1971)
Your Name. (2016)
Winnie the Pooh and the Day of Concern (1972)
Comedy Central Roast of David Hasselhoff (2010)
DC Super Hero Girls: Hero of the Year (2016)
Hacksaw Ridge (2016)
David Cross: Making America Great Again (2016)
Over the Garden Wall (2013)
Wings, Legs and Tails (1986)
Blair Witch (2016)
31 (2016)
ARQ (2016)
Arrival (2016)
Storks (2016)
Maximum Ride (2016)
Endless Poetry (2016)
The Girl with All the Gifts (2016)
All Roads Lead to Rome (2016)
Amanda Knox (2016)
Dirty 30 (2016)
Gimme Danger (2016)
Go Figure (2005)
Anything for Love (2016)
Night Guards (2016)
La La Land (2016)
13th (2016)
London Town (2016)
Inferno (2016)
Keeping Up with the Joneses (2016)
Wild Oats (2016)
The Rocky Horror Pict

In [14]:
## mapping movie id to the movie name using join function
movie_csv = spark.read.option("header","true").option("inferschema","true").\
csv("./data/movies.csv")

movie_csv.printSchema()

#can create table from df and perform SQL queries
movie_csv.createOrReplaceTempView("moviename_table")       

#find out average numb of friends of diff age group using SQL query on table


popular_movie_average_rating.createOrReplaceTempView("rating_movieid")       

popular_movie_average_rating.printSchema()

#mapping moviid to name using join/inner
popular_movie_average_rating.join(movie_csv,popular_movie_average_rating.movieId ==  movie_csv.movieId,"inner") \
     .show(truncate=True)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- Movie_id_count: long (nullable = false)
 |-- Average_rating: double (nullable = true)

+-------+--------------+------------------+-------+--------------------+--------------------+
|movieId|Movie_id_count|    Average_rating|movieId|               title|              genres|
+-------+--------------+------------------+-------+--------------------+--------------------+
|   1580|           165| 3.487878787878788|   1580|Men in Black (a.k...|Action|Comedy|Sci-Fi|
|   2366|            25|              3.64|   2366|    King Kong (1933)|Action|Adventure|...|
|   3175|            75|              3.58|   3175| Galaxy Quest (1999)|Adventure|Comedy|...|
|   1088|            42| 3.369047619047619|   1088|Dirty Dancing (1987)|Drama|Musical|Rom...|
|  32460|             4|              4.25|  32460|Knockin' on Heave...|Action|Comed

In [15]:
#mapping using SQL query;implicit join where clause


moviename_id_map = spark.sql("SELECT rating_movieid.movieId,Average_rating,\
                            Movie_id_count, title from rating_movieid,\
                           moviename_table where moviename_table.movieId==rating_movieid.movieId order by Movie_id_count desc  ")
moviename_id_map.show(4)               



+-------+-----------------+--------------+--------------------+
|movieId|   Average_rating|Movie_id_count|               title|
+-------+-----------------+--------------+--------------------+
|    356|4.164133738601824|           329| Forrest Gump (1994)|
|    318|4.429022082018927|           317|Shawshank Redempt...|
|    296|4.197068403908795|           307| Pulp Fiction (1994)|
|    593|4.161290322580645|           279|Silence of the La...|
+-------+-----------------+--------------+--------------------+
only showing top 4 rows



In [16]:
 schema = StructType([StructField("Id",IntegerType(),True),
    StructField("superhero",StringType(),True)\
])

superhero_df = spark.read.schema(schema).option("sep"," ").csv("./data/Marvel+Names.txt")
#type(superhero_df) --> pyspark.sql.dataframe.DataFrame

suphero_id_df = spark.read.text("./data/Marvel+Graph.txt")
suphero_id_df.show(2)


+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
|5989 4080 4264 44...|
+--------------------+
only showing top 2 rows



In [17]:
spark.stop()